# Declare necessary libraries

In [157]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss

# Loading and standardization data

In [158]:

import pandas as pd

def load_prepare_data(file_path):
    # Đọc dòng đầu tiên để xác định dấu phân tách
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline()
    
    # Xác định dấu phân tách (';' hoặc ',')
    sep = ';' if ';' in first_line else ','
    
    # Đọc file CSV với dấu phân tách đã xác định
    try:
        df = pd.read_csv(file_path, sep=sep)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None
    
    print(f"Processing file: {file_path}")
    print("Columns:", df.columns.tolist())
    
    # Định nghĩa các cột cần giữ cho từng loại file
    columns_to_keep = []
    
    # Trường hợp 1: Dữ liệu có cột 'timeOpen', 'timeClose', 'name', 'close', ...
    if {'timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'timestamp'}.issubset(df.columns):
        columns_to_keep = ['timeClose', 'close']
    
    # Trường hợp 2: Dữ liệu có cột 'Date', 'Open', 'High', 'Low', 'Close', ...
    elif {'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'}.issubset(df.columns):
        columns_to_keep = ['Date', 'Close']
    
    # Trường hợp 3: Dữ liệu có cột 'Date', 'Price', 'Open', ...
    elif {'Date', 'Price', 'Open', 'High', 'Low', 'Change %'}.issubset(df.columns):
        columns_to_keep = ['Date', 'Price']
    
    # Lọc các cột cần thiết nếu tìm thấy
    if columns_to_keep:
        df = df[columns_to_keep]
    else:
        print(f"No recognized columns to keep for file: {file_path}")
        return None
    
    # **Xử lý cột giá cả ('Price', 'Close', 'close')**
    for col in ['Price', 'Close', 'close']:
        if col in df.columns:
            # Loại bỏ các ký tự không cần thiết và chuyển đổi sang kiểu số
            df[col] = df[col].replace({'"': '', ',': ''}, regex=True)
            df[col] = df[col].replace({r'[^0-9.\-]': ''}, regex=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
     # Xử lý cột thời gian ('Date', 'timeClose') và các cột thời gian khác nếu có
    if 'Date' in df.columns:
        try:
            df['Date'] = pd.to_datetime(df['Date'], utc=True, errors='coerce').dt.tz_convert(None).dt.date
        except Exception as e:
            print(f"Error converting 'Date' in {file_path}: {e}")
    
    if 'timeClose' in df.columns:
        try:
            df['timeClose'] = pd.to_datetime(df['timeClose'], utc=True, errors='coerce').dt.tz_convert(None).dt.date
        except Exception as e:
            print(f"Error converting 'timeClose' in {file_path}: {e}")
    
    return df



In [159]:
Bitcoin = load_prepare_data(r'1stJan2017 - 11thNov2024\crypto - format 1\Bitcoin_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv')
SP500 = load_prepare_data(r'1stJan2017 - 11thNov2024/format 2/SP500.csv')
Gold = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\XAU_USD Historical Data.csv')
Silver = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\XAG_USD Historical Data (1).csv')
IMUS = load_prepare_data(r'1stJan2017 - 11thNov2024\format 2\IMUS.csv')
WTI = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\WTI_USD Historical Data (1).csv')
Dollar = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\US Dollar Index Historical Data (1).csv')
Franc = load_prepare_data(r'1stJan2017 - 11thNov2024\format 3\CHF_USD Historical Data (1).csv')
Ethereum = load_prepare_data(r'1stJan2017 - 11thNov2024\crypto - format 1\Ethereum_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv')
GSCI=load_prepare_data(r'1stJan2017 - 11thNov2024\format 2\GSCI.csv')

Processing file: 1stJan2017 - 11thNov2024\crypto - format 1\Bitcoin_11_21_2016-1_20_2017_historical_data_coinmarketcap.csv
Columns: ['timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high', 'low', 'close', 'volume', 'marketCap', 'timestamp']
Processing file: 1stJan2017 - 11thNov2024/format 2/SP500.csv
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
Processing file: 1stJan2017 - 11thNov2024\format 3\XAU_USD Historical Data.csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: 1stJan2017 - 11thNov2024\format 3\XAG_USD Historical Data (1).csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']
Processing file: 1stJan2017 - 11thNov2024\format 2\IMUS.csv
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
Processing file: 1stJan2017 - 11thNov2024\format 3\WTI_USD Historical Data (1).csv
Columns: ['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', '

In [160]:
Bitcoin = Bitcoin.rename(columns={'timeClose': 'Date'})
Ethereum = Ethereum.rename(columns={'timeClose': 'Date'})


In [161]:
Bitcoin = Bitcoin.rename(columns={'close': 'Price'})
Ethereum = Ethereum.rename(columns={'close': 'Price'})


In [162]:
SP500=SP500.rename(columns={'Close': 'Price'})
IMUS=IMUS.rename(columns={'Close': 'Price'})
GSCI=GSCI.rename(columns={'Close': 'Price'})

# Segment data by phase

In [163]:
# Tạo dictionary cho tất cả các tài sản
assets = {
    "SP500": SP500,
    "Gold": Gold,
    "Silver": Silver,
    "GSCI":GSCI,
    "IMUS": IMUS,
    "WTI": WTI,
    "Dollar": Dollar,
    "Franc": Franc,
    "Bitcoin": Bitcoin,
    "Ethereum": Ethereum,
}

Hàm tạo cột chỉ mục ( index) bằng ngày tháng thay vì số thứ tự . Điều này giúp cho việc kết hợp , phân chia dữ liệu được đồng về mặt thời gian

In [164]:
def convert_to_datetime_index_with_copy(assets, date_column='Date', copy_column_name='Time'):
    for name, df in assets.items():
        if date_column in df.columns:
            try:
                # Chuyển đổi cột 'Date' sang kiểu datetime
                df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
                
                # Loại bỏ các dòng có giá trị null trong cột 'Date' trước khi sao chép
                df.dropna(subset=[date_column], inplace=True)
                
                # Tạo một cột bản sao 'Time' từ cột 'Date'
                df[copy_column_name] = df[date_column]
                
                # Đặt cột 'Time' làm index
                df.set_index(copy_column_name, inplace=True)
                
                print(f"Đã chuyển '{date_column}' thành datetime và tạo index từ cột '{copy_column_name}' cho bộ dữ liệu {name}")
            except Exception as e:
                print(f"Lỗi khi xử lý bộ dữ liệu {name}: {e}")
        else:
            print(f"Bộ dữ liệu {name} không có cột '{date_column}'")
    return assets

# Sử dụng hàm với tập dữ liệu assets
assets = convert_to_datetime_index_with_copy(assets)

Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu SP500
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Gold
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Silver
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu GSCI
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu IMUS
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu WTI
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Dollar
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Franc
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Bitcoin
Đã chuyển 'Date' thành datetime và tạo index từ cột 'Time' cho bộ dữ liệu Ethereum


hàm phân chia dữ liệu theo giai đoạn

In [165]:
def segment_data_by_period(assets, periods, date_column='Date'):
    # Khởi tạo từ điển để lưu dữ liệu đã phân đoạn
    segmented_data = {period_name: {} for period_name in periods}

    for period_name, (start_date, end_date) in periods.items():
        # Chuyển start_date và end_date sang datetime và normalize
        start_date = pd.to_datetime(start_date).normalize()
        end_date = pd.to_datetime(end_date).normalize()

        for name, df in assets.items():
            # Chuyển cột 'Date' thành datetime và normalize nếu cần
            if date_column in df.columns:
                df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()

            # Kiểm tra và in các giá trị min, max để xác nhận phạm vi thời gian
            print(f"Kiểm tra dữ liệu {name} trong giai đoạn {period_name}: {df[date_column].min()} - {df[date_column].max()}")

            # Tạo dải ngày đầy đủ cho giai đoạn hiện tại
            full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
            
            # Tạo DataFrame mới với dải ngày đầy đủ
            df_period = pd.DataFrame({date_column: full_date_range})
            
            # Hợp nhất dữ liệu gốc với dải ngày đầy đủ dựa trên cột 'Date'
            if date_column in df.columns:
                df_merged = pd.merge(df_period, df, on=date_column, how='left')
                segmented_data[period_name][name] = df_merged
                print(f"Đã hợp nhất dữ liệu cho {name} trong giai đoạn {period_name} với dải ngày đầy đủ")
            else:
                print(f"Dữ liệu {name} không có cột {date_column}, bỏ qua.")

    return segmented_data


In [166]:

covid_period = ('2020-01-05', '2023-05-07')
full_sample_period = ('2017-01-01', '2024-11-11')
before_crisic= ('2017-01-01', '2020-01-04')
after_crisic=('2022-07-08', '2024-11-11')
periods = {
    "Full sample": full_sample_period,
    "COVID-19": covid_period,
}

# Phân đoạn dữ liệu
segmented_data = segment_data_by_period(assets, periods)

Kiểm tra dữ liệu SP500 trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho SP500 trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu Gold trong giai đoạn Full sample: 2017-01-02 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho Gold trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu Silver trong giai đoạn Full sample: 2017-01-02 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho Silver trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu GSCI trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho GSCI trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu IMUS trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho IMUS trong giai đoạn Full sample với dải ngày đầy đủ
Kiểm tra dữ liệu WTI trong giai đoạn Full sample: 2017-01-03 00:00:00 - 2024-11-11 00:00:00
Đã hợp nhất dữ liệu cho WTI trong giai đoạn Full sam

hàm kiểm tra dữ liệu sau khi phân chia

In [167]:
def check_segmented_data(segmented_data, periods, date_column='Date'):
    # Duyệt qua từng giai đoạn và tài sản đã phân đoạn
    for period_name, assets_in_period in segmented_data.items():
        print(f"Giai đoạn: {period_name}")
        start_date, end_date = periods[period_name]  # Lấy mốc thời gian của từng giai đoạn
        start_date = pd.to_datetime(start_date).normalize()
        end_date = pd.to_datetime(end_date).normalize()

        for asset_name, df in assets_in_period.items():
            if not df.empty:  # Kiểm tra nếu DataFrame không rỗng
                print(f"  Tài sản: {asset_name}")
                
                # Kiểm tra nếu cột 'Date' tồn tại
                if date_column in df.columns:
                    # Chuyển cột 'Date' thành datetime và chuẩn hóa
                    df[date_column] = pd.to_datetime(df[date_column]).dt.normalize()
                    
                    # Lấy ngày bắt đầu và kết thúc từ cột 'Date'
                    min_date = df[date_column].min()
                    max_date = df[date_column].max()
                    
                    print(f"    Bắt đầu: {min_date}, Kết thúc: {max_date}")
                    
                    # Kiểm tra xem dữ liệu có nằm trong phạm vi giai đoạn không
                    if min_date >= start_date and max_date <= end_date:
                        print(f"    Dữ liệu hợp lệ trong phạm vi {start_date} đến {end_date}")
                    else:
                        print(f"    Lỗi: Dữ liệu không nằm trong phạm vi {start_date} đến {end_date}")
                else:
                    print(f"    Lỗi: Tài sản {asset_name} không có cột '{date_column}'")
            else:
                print(f"  Tài sản: {asset_name} - Không có dữ liệu trong giai đoạn này")
        print("-" * 40)


In [168]:
# Kiểm tra dữ liệu đã phân đoạn
check_segmented_data(segmented_data, periods)

Giai đoạn: Full sample
  Tài sản: SP500
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: Gold
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: Silver
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: GSCI
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: IMUS
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: WTI
    Bắt đầu: 2017-01-01 00:00:00, Kết thúc: 2024-11-11 00:00:00
    Dữ liệu hợp lệ trong phạm vi 2017-01-01 00:00:00 đến 2024-11-11 00:00:00
  Tài sản: Dollar
    Bắt đầu: 20

In [169]:
segmented_data['COVID-19']['Bitcoin']

,Date,Price
0,2020-01-05,7411.317327
1,2020-01-06,7769.219039
2,2020-01-07,8163.692239
3,2020-01-08,8079.862777
4,2020-01-09,7879.071524
...,...,...
1214,2023-05-03,29006.307671
1215,2023-05-04,28847.711591
1216,2023-05-05,29534.384020
1217,2023-05-06,28904.623949


# Calculate daily profit for each period

Xử lý giá trị Price

In [170]:
def handle_missing_values(df, price_column='Price'):

    # Kiểm tra nếu cột 'Price' có trong DataFrame
    if price_column in df.columns:
        # Đầu tiên, xác định các khoảng thời gian liên tiếp bị thiếu
        df['Missing'] = df[price_column].isna()
        df['Missing_Group'] = (df['Missing'] != df['Missing'].shift()).cumsum()
        
        # Đếm số lượng ngày liên tiếp bị thiếu
        missing_counts = df.groupby('Missing_Group')['Missing'].transform('sum')
        
        # Sử dụng nội suy cho các khoảng thời gian bị thiếu dưới 3 ngày
        df.loc[(df['Missing']) & (missing_counts < 4), price_column] = df[price_column].interpolate(method='linear')
        
        # Xóa các cột tạm thời
        df.drop(['Missing', 'Missing_Group'], axis=1, inplace=True)
    
    return df

In [171]:

def log_returns(segmented_data, price_column='Price'):
    """
    Tính toán log return sau khi xử lý missing values cho từng giai đoạn và từng tài sản.
    """
    # Tạo dictionary để lưu trữ log returns cho từng giai đoạn
    log_returns_data = {period: {} for period in segmented_data}

    # Duyệt qua từng giai đoạn và từng tài sản
    for period, assets in segmented_data.items():
        for asset_name, df in assets.items():
            if price_column in df.columns:
                try:
                    # Xử lý missing values cho cột 'Price'
                    df = handle_missing_values(df, price_column)
                    
                    # Tính log return
                    df['Log Return'] = (np.log(df[price_column] / df[price_column].shift(1)))*100
                    
                    # Lưu DataFrame đã tính log return vào dictionary
                    log_returns_data[period][asset_name] = df
                    print(f"Tính Log Return cho {asset_name} trong giai đoạn {period} thành công.")
                except Exception as e:
                    print(f"Lỗi khi tính toán Log Return cho {asset_name} trong giai đoạn {period}: {e}")
            else:
                print(f"Tài sản {asset_name} không có cột {price_column}. Bỏ qua.")

    return log_returns_data

In [172]:
log_returns_data = log_returns(segmented_data)

# Kiểm tra kết quả
for period, assets in log_returns_data.items():
    for asset, df in assets.items():
        print(f"{asset} - {period}:")
        print(df[['Date', 'Price', 'Log Return']].head())

Tính Log Return cho SP500 trong giai đoạn Full sample thành công.
Tính Log Return cho Gold trong giai đoạn Full sample thành công.
Tính Log Return cho Silver trong giai đoạn Full sample thành công.
Tính Log Return cho GSCI trong giai đoạn Full sample thành công.
Tính Log Return cho IMUS trong giai đoạn Full sample thành công.
Tính Log Return cho WTI trong giai đoạn Full sample thành công.
Tính Log Return cho Dollar trong giai đoạn Full sample thành công.
Tính Log Return cho Franc trong giai đoạn Full sample thành công.
Tính Log Return cho Bitcoin trong giai đoạn Full sample thành công.
Tính Log Return cho Ethereum trong giai đoạn Full sample thành công.
Tính Log Return cho SP500 trong giai đoạn COVID-19 thành công.
Tính Log Return cho Gold trong giai đoạn COVID-19 thành công.
Tính Log Return cho Silver trong giai đoạn COVID-19 thành công.
Tính Log Return cho GSCI trong giai đoạn COVID-19 thành công.
Tính Log Return cho IMUS trong giai đoạn COVID-19 thành công.
Tính Log Return cho WTI t

# Xử lý giá trị lợi nhuận 

In [173]:
from statsmodels.robust.robust_linear_model import RLM
from statsmodels.robust.norms import HuberT
import statsmodels.api as sm
import numpy as np
import pandas as pd

def preprocess_log_returns(log_returns_data, columns_to_process=None, remove_outliers=False):
    # Tạo dictionary để lưu trữ dữ liệu đã xử lý cho từng giai đoạn
    processed_data = {period: {} for period in log_returns_data}

    # Duyệt qua từng giai đoạn và từng tài sản trong log_returns_data
    for period, assets in log_returns_data.items():
        print(f"Đang xử lý log returns cho giai đoạn: {period}")
        for asset_name, asset_data in assets.items():
            # Kiểm tra sự tồn tại của cột 'Date' và các cột quan trọng khác
            required_columns = ['Log Return']
            if not all(col in asset_data.columns for col in required_columns + ['Date']):
                print(f"Tài sản {asset_name} không có đủ các cột cần thiết. Bỏ qua.")
                continue

            # Tạo bản sao của DataFrame để xử lý
            series = asset_data.copy()

            # Nếu `columns_to_process` không được cung cấp, mặc định là các cột bắt buộc
            if columns_to_process is None:
                columns_to_process = required_columns

            # Duyệt qua các cột cần xử lý
            for column in columns_to_process:
                if column not in series.columns:
                    print(f"Tài sản {asset_name} không có cột '{column}'. Bỏ qua.")
                    continue

                # 1. Thay giá trị 0 thành NaN
                series[column] = series[column].replace(0, np.nan)

                # 2. Nội suy tuyến tính để điền các giá trị thiếu (NaN)
                series[column] = series[column].interpolate(method='linear')

                # 3. Điền giá trị thiếu bằng giá trị trung vị (median) để tránh bias
                series[column] = series[column].fillna(series[column].median())

                # 4. Xử lý outliers (nếu remove_outliers = True)
                if remove_outliers:
                    # Fit model Huber's M-estimator cho cột dữ liệu
                    try:
                        model = RLM(series[column].dropna(), sm.add_constant(range(len(series[column].dropna()))), M=HuberT())
                        result = model.fit()

                        # Dự đoán giá trị và tính residuals
                        residuals = result.resid
                        threshold = residuals.std() * 3  # Ngưỡng là 3 lần độ lệch chuẩn
                        series[column] = series[column] - np.where(np.abs(residuals) > threshold, residuals, 0)
                    except Exception as e:
                        print(f"Lỗi khi xử lý outliers cho tài sản {asset_name}, cột {column}: {e}")

                # 5. Làm mượt dữ liệu bằng EMA
                ema_span = 5  
                series[column] = series[column].ewm(span=ema_span, adjust=False).mean()

            # Bỏ các hàng có NaN còn lại sau khi nội suy và xử lý outliers
            series.dropna(subset=columns_to_process, inplace=True)

            # Kiểm tra nếu dữ liệu sau khi xử lý còn đủ điểm dữ liệu
            if series.empty:
                print(f"Tài sản {asset_name} sau khi xử lý không có dữ liệu. Bỏ qua.")
                continue

            # Lưu lại DataFrame đã xử lý vào dictionary
            processed_data[period][asset_name] = series
            print(f"Đã xử lý log returns cho {asset_name} trong giai đoạn {period} thành công.")

    return processed_data


In [174]:
Log_return_cleared = preprocess_log_returns(
    log_returns_data=log_returns_data,
    columns_to_process=['Log Return'], 
    remove_outliers=True  
)


Đang xử lý log returns cho giai đoạn: Full sample
Đã xử lý log returns cho SP500 trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Gold trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Silver trong giai đoạn Full sample thành công.
Đã xử lý log returns cho GSCI trong giai đoạn Full sample thành công.
Đã xử lý log returns cho IMUS trong giai đoạn Full sample thành công.
Đã xử lý log returns cho WTI trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Dollar trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Franc trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Bitcoin trong giai đoạn Full sample thành công.
Đã xử lý log returns cho Ethereum trong giai đoạn Full sample thành công.
Đang xử lý log returns cho giai đoạn: COVID-19
Đã xử lý log returns cho SP500 trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho Gold trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho Silver trong giai đoạn COVID-19

In [175]:
Log_return_cleared['Full sample']['SP500']

,Date,Price,Log Return
0,2017-01-01,NaN,0.032404
1,2017-01-02,NaN,0.032404
2,2017-01-03,2257.830078,0.032404
3,2017-01-04,2270.750000,0.211802
4,2017-01-05,2269.000000,0.115502
...,...,...,...
2867,2024-11-07,5973.100098,0.935908
2868,2024-11-08,5995.540039,0.748932
2869,2024-11-09,5997.476725,0.510054
2870,2024-11-10,5999.413411,0.350798


*  dữ liệu cuối  trước khi đưua vào model : Log_return_cleared

#  Descriptive statistics

In [176]:
from scipy.stats import skew, kurtosis, jarque_bera

In [177]:
def generate_summary_table(Log_return_cleared):
    """
    Hàm này tạo bảng thống kê bao gồm các thống kê tổng hợp và kiểm định Jarque-Bera.
    Các ký hiệu *, **, và *** tương ứng với mức ý nghĩa 1%, 5%, và 10%.
    """
    data_list = []

    # Duyệt qua từng giai đoạn và từng tài sản trong Log_return_cleared
    for period, data_dict in Log_return_cleared.items():
        print(f"Đang tạo bảng tóm tắt cho giai đoạn: {period}")
        for asset, df in data_dict.items():
            # Kiểm tra nếu cột 'Log Return' tồn tại và loại bỏ các giá trị NaN
            if 'Log Return' in df.columns:
                df_cleaned = df['Log Return'].dropna()

                # Kiểm tra nếu còn đủ dữ liệu để tính toán
                if len(df_cleaned) > 0:
                    # Tính các thống kê cho từng tài sản và từng giai đoạn
                    mean = df_cleaned.mean()
                    std_dev = df_cleaned.std()
                    sharpe_ratio = mean / std_dev if std_dev != 0 else 0
                    skewness = df_cleaned.skew()
                    kurtosis = df_cleaned.kurtosis()

                    # Kiểm định Jarque-Bera
                    jb_stat, jb_p_value = jarque_bera(df_cleaned)

                    # Đánh dấu mức ý nghĩa thống kê
                    if jb_p_value < 0.01:
                        jb_significance = '*'
                    elif jb_p_value < 0.05:
                        jb_significance = '**'
                    elif jb_p_value < 0.10:
                        jb_significance = '***'
                    else:
                        jb_significance = ''

                    # Lưu kết quả vào danh sách
                    data_list.append({
                        'Variable': asset,
                        'Period': period,
                        'Mean': mean,
                        'Std. Dev.': std_dev,
                        'Sharpe Ratio': sharpe_ratio,
                        'Skewness': skewness,
                        'Kurtosis': kurtosis,
                        'Jarque-Bera': f"{jb_stat:.4f}{jb_significance}"
                    })

    # Chuyển kết quả thành DataFrame
    summary_df = pd.DataFrame(data_list)

    return summary_df

# Sử dụng hàm generate_summary_table với dữ liệu đã được xử lý
summary_df = generate_summary_table(Log_return_cleared)

Đang tạo bảng tóm tắt cho giai đoạn: Full sample
Đang tạo bảng tóm tắt cho giai đoạn: COVID-19


In [178]:
summary_df

,Variable,Period,Mean,Std. Dev.,Sharpe Ratio,Skewness,Kurtosis,Jarque-Bera
0,SP500,Full sample,0.058700,0.319212,0.183889,-0.357459,1.913646,496.8450*
1,Gold,Full sample,0.035216,0.267050,0.131869,0.128787,0.710610,67.7944*
2,Silver,Full sample,0.034503,0.513669,0.067170,-0.048554,1.041228,129.8943*
3,GSCI,Full sample,0.000313,0.149142,0.002098,0.109799,0.416839,26.2775*
4,IMUS,Full sample,0.000313,0.149142,0.002098,0.109799,0.416839,26.2775*
5,WTI,Full sample,0.047672,0.802554,0.059400,-0.419417,1.552425,370.7374*
6,Dollar,Full sample,0.006516,0.140528,0.046367,-0.061995,1.350475,218.6516*
7,Franc,Full sample,0.000521,0.145972,0.003566,0.057860,0.415214,21.9532*
8,Bitcoin,Full sample,0.201880,1.442663,0.139936,0.137523,1.017993,132.1164*
9,Ethereum,Full sample,0.197897,1.861482,0.106312,0.032238,1.486381,263.2194*


#  unit-root tests

In [179]:
from statsmodels.tsa.stattools import adfuller
from arch.unitroot import PhillipsPerron

def run_unit_root_tests(Log_return_cleared):
    data_list = []

    # Duyệt qua từng giai đoạn và từng tài sản
    for period, data_dict in Log_return_cleared.items():
        for asset, df in data_dict.items():
            if 'Log Return' in df.columns:
                
                # Kiểm tra nếu còn đủ dữ liệu để tính toán và không chứa NaN/Inf
                log_returns = df['Log Return'].dropna()
                
                if len(log_returns) > 0:
                    # Kiểm định ADF
                    adf_stat, adf_p_value, _, _, _, _ = adfuller(log_returns)
                    
                    # Đánh dấu mức ý nghĩa ADF
                    if adf_p_value < 0.01:
                        adf_significance = '*'
                    elif adf_p_value < 0.05:
                        adf_significance = '**'
                    elif adf_p_value < 0.10:
                        adf_significance = '***'
                    else:
                        adf_significance = ''
                    
                    # Kiểm định PP
                    pp_result = PhillipsPerron(log_returns)
                    pp_stat, pp_p_value = pp_result.stat, pp_result.pvalue
                    
                    # Đánh dấu mức ý nghĩa PP
                    if pp_p_value < 0.01:
                        pp_significance = '*'
                    elif pp_p_value < 0.05:
                        pp_significance = '**'
                    elif pp_p_value < 0.10:
                        pp_significance = '***'
                    else:
                        pp_significance = ''
                    
                    # Lưu kết quả vào list
                    data_list.append({
                        'Variable': asset,
                        'Period': period,
                        'ADF': f"{adf_stat:.4f}{adf_significance}",
                        'ADF p-value': adf_p_value,
                        'PP': f"{pp_stat:.4f}{pp_significance}",
                        'PP p-value': pp_p_value
                    })

    # Chuyển kết quả thành DataFrame
    summary_df = pd.DataFrame(data_list)


    
    return summary_df

# Ví dụ sử dụng hàm
summary_unit_root_df = run_unit_root_tests(log_returns_data)

In [180]:
summary_unit_root_df

,Variable,Period,ADF,ADF p-value,PP,PP p-value
0,SP500,Full sample,-11.9735*,3.848720e-22,-55.7762*,0.0
1,Gold,Full sample,-51.9704*,0.000000e+00,-51.9486*,0.0
2,Silver,Full sample,-12.2977*,7.571353e-23,-51.1328*,0.0
3,GSCI,Full sample,-48.8937*,0.000000e+00,-48.6979*,0.0
4,IMUS,Full sample,-48.8937*,0.000000e+00,-48.6979*,0.0
5,WTI,Full sample,-10.2273*,5.137502e-18,-46.3869*,0.0
6,Dollar,Full sample,-50.9190*,0.000000e+00,-50.8544*,0.0
7,Franc,Full sample,-49.3887*,0.000000e+00,-49.2271*,0.0
8,Bitcoin,Full sample,-36.8925*,0.000000e+00,-55.1842*,0.0
9,Ethereum,Full sample,-15.6037*,1.816056e-28,-55.8428*,0.0


# Correlation matrix

In [181]:
def create_correlation_matrix(Log_return_cleared, period, column='Log Return'):

    # Kiểm tra nếu giai đoạn có tồn tại trong dữ liệu
    if period not in Log_return_cleared:
        print(f"Giai đoạn '{period}' không có trong dữ liệu.")
        return None
    
    # Tạo DataFrame chứa log return của tất cả các tài sản trong giai đoạn đã chọn
    assets_data = {}
    for asset_name, df in Log_return_cleared[period].items():
        if column in df.columns:
            assets_data[asset_name] = df[column].dropna()  # Lấy cột Log Return và loại bỏ NaN
    
    # Tạo DataFrame từ dictionary assets_data
    correlation_df = pd.DataFrame(assets_data)
    
    # Kiểm tra nếu DataFrame rỗng
    if correlation_df.empty:
        print("Không có dữ liệu cho các tài sản trong giai đoạn này.")
        return None

    # Tính toán ma trận tương quan cho tất cả các tài sản
    correlation_matrix = correlation_df.corr()

    

    return correlation_matrix

In [182]:
correlation_matrix_full = create_correlation_matrix(
    Log_return_cleared, 
    period='Full sample', 
    column='Log Return', 
)

In [183]:
correlation_matrix_full

,SP500,Gold,Silver,GSCI,IMUS,WTI,Dollar,Franc,Bitcoin,Ethereum
SP500,1.000000,0.061204,0.185601,0.091240,0.091240,0.194860,-0.208485,0.106822,0.183869,0.212502
Gold,0.061204,1.000000,0.712990,0.388657,0.388657,0.097501,-0.387562,0.414114,0.107201,0.125263
Silver,0.185601,0.712990,1.000000,0.317387,0.317387,0.164812,-0.377277,0.333995,0.136526,0.129885
GSCI,0.091240,0.388657,0.317387,1.000000,1.000000,-0.006831,-0.692811,0.906480,0.081358,0.116669
IMUS,0.091240,0.388657,0.317387,1.000000,1.000000,-0.006831,-0.692811,0.906480,0.081358,0.116669
WTI,0.194860,0.097501,0.164812,-0.006831,-0.006831,1.000000,-0.076138,-0.004932,0.037213,0.069504
Dollar,-0.208485,-0.387562,-0.377277,-0.692811,-0.692811,-0.076138,1.000000,-0.718054,-0.137028,-0.177900
Franc,0.106822,0.414114,0.333995,0.906480,0.906480,-0.004932,-0.718054,1.000000,0.091924,0.127940
Bitcoin,0.183869,0.107201,0.136526,0.081358,0.081358,0.037213,-0.137028,0.091924,1.000000,0.672121
Ethereum,0.212502,0.125263,0.129885,0.116669,0.116669,0.069504,-0.177900,0.127940,0.672121,1.000000


# Merge Data

In [184]:
def merge_assets_with_sp500(Log_return_cleared, periods=None, assets_to_merge=None):
    if assets_to_merge is None:
        assets_to_merge = [ 'Gold', 'Silver', 'Tbond', 
                           'Dollar', 'IMUS', 'WTI','GSCI', 'Franc','Bitcoin', 
                           'Ethereum']

    # Tạo dictionary để lưu kết quả merge cho từng giai đoạn
    merged_data_dict = {}

    # Xử lý trường hợp periods là một danh sách hoặc một chuỗi
    period_list = [periods] if isinstance(periods, str) else periods

    for period in period_list:
        # Kiểm tra nếu giai đoạn tồn tại trong Log_return_cleared
        if period not in Log_return_cleared:
            print(f"Giai đoạn {period} không tồn tại trong Log_return_cleared. Bỏ qua.")
            continue

        assets_data = Log_return_cleared[period]

        # Kiểm tra nếu S&P 500 có trong dữ liệu
        if 'SP500' not in assets_data:
            print(f"Không tìm thấy dữ liệu S&P 500 cho giai đoạn {period}. Bỏ qua.")
            continue

        # Chỉ giữ lại cột 'Date' và 'Log Return' từ S&P 500
        sp500_log_return = assets_data['SP500'][['Date', 'Log Return']].rename(columns={'Log Return': 'SP500_Log_Return'})

        # Tạo dictionary cho từng giai đoạn nếu chưa có
        if period not in merged_data_dict:
            merged_data_dict[period] = {}

        # Duyệt qua các tài sản cần merge
        for asset in assets_to_merge:
            if asset in assets_data:
                try:
                    asset_data = assets_data[asset]

                    # Kiểm tra nếu cột 'Log Return' tồn tại trong dữ liệu tài sản
                    if 'Log Return' not in asset_data.columns:
                        print(f"Tài sản {asset} không có cột 'Log Return' trong giai đoạn {period}. Bỏ qua.")
                        continue

                    # Thực hiện merge chỉ với cột 'SP500_Log_Return' dựa trên 'Date'
                    merged_data = pd.merge(asset_data, sp500_log_return, on='Date', how='left')

                    # Lưu kết quả vào dictionary
                    merged_data_dict[period][asset.lower()] = merged_data

                    print(f"Merge thành công cho {asset} trong giai đoạn {period}.")
                except Exception as e:
                    print(f"Lỗi khi merge {asset} trong giai đoạn {period}: {e}")
            else:
                print(f"Tài sản {asset} không tồn tại trong giai đoạn {period}.")

    return merged_data_dict



In [185]:
merged_data_crisis=merge_assets_with_sp500(Log_return_cleared,periods='COVID-19')

Merge thành công cho Gold trong giai đoạn COVID-19.
Merge thành công cho Silver trong giai đoạn COVID-19.
Tài sản Tbond không tồn tại trong giai đoạn COVID-19.
Merge thành công cho Dollar trong giai đoạn COVID-19.
Merge thành công cho IMUS trong giai đoạn COVID-19.
Merge thành công cho WTI trong giai đoạn COVID-19.
Merge thành công cho GSCI trong giai đoạn COVID-19.
Merge thành công cho Franc trong giai đoạn COVID-19.
Merge thành công cho Bitcoin trong giai đoạn COVID-19.
Merge thành công cho Ethereum trong giai đoạn COVID-19.


In [186]:
merged_data=merge_assets_with_sp500(Log_return_cleared,periods='Full sample')

Merge thành công cho Gold trong giai đoạn Full sample.
Merge thành công cho Silver trong giai đoạn Full sample.
Tài sản Tbond không tồn tại trong giai đoạn Full sample.
Merge thành công cho Dollar trong giai đoạn Full sample.
Merge thành công cho IMUS trong giai đoạn Full sample.
Merge thành công cho WTI trong giai đoạn Full sample.
Merge thành công cho GSCI trong giai đoạn Full sample.
Merge thành công cho Franc trong giai đoạn Full sample.
Merge thành công cho Bitcoin trong giai đoạn Full sample.
Merge thành công cho Ethereum trong giai đoạn Full sample.


In [187]:
merger_result_crisis=preprocess_log_returns(merged_data_crisis,columns_to_process=['Log Return', 'SP500_Log_Return'])

Đang xử lý log returns cho giai đoạn: COVID-19
Đã xử lý log returns cho gold trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho silver trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho dollar trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho imus trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho wti trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho gsci trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho franc trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho bitcoin trong giai đoạn COVID-19 thành công.
Đã xử lý log returns cho ethereum trong giai đoạn COVID-19 thành công.


In [188]:
merger_result_crisis['COVID-19']['bitcoin']

,Date,Price,Log Return,SP500_Log_Return
0,2020-01-05,7411.317327,0.071455,0.029580
1,2020-01-06,7769.219039,0.587532,0.029580
2,2020-01-07,8163.692239,1.301917,-0.004898
3,2020-01-08,8079.862777,1.360079,0.034662
4,2020-01-09,7879.071524,0.955200,0.122095
...,...,...,...,...
1214,2023-05-03,29006.307671,0.065053,-0.133805
1215,2023-05-04,28847.711591,0.055729,-0.265829
1216,2023-05-05,29534.384020,0.306775,-0.091569
1217,2023-05-06,28904.623949,0.144782,0.199457


In [189]:
merge_result=preprocess_log_returns(merged_data,columns_to_process=['Log Return', 'SP500_Log_Return'])

Đang xử lý log returns cho giai đoạn: Full sample
Đã xử lý log returns cho gold trong giai đoạn Full sample thành công.
Đã xử lý log returns cho silver trong giai đoạn Full sample thành công.
Đã xử lý log returns cho dollar trong giai đoạn Full sample thành công.
Đã xử lý log returns cho imus trong giai đoạn Full sample thành công.
Đã xử lý log returns cho wti trong giai đoạn Full sample thành công.
Đã xử lý log returns cho gsci trong giai đoạn Full sample thành công.
Đã xử lý log returns cho franc trong giai đoạn Full sample thành công.
Đã xử lý log returns cho bitcoin trong giai đoạn Full sample thành công.
Đã xử lý log returns cho ethereum trong giai đoạn Full sample thành công.


In [190]:
merged_data['Full sample']['bitcoin']

,Date,Price,Log Return,SP500_Log_Return
0,2017-01-01,998.325012,0.136812,0.032404
1,2017-01-02,1021.750000,0.864316,0.032404
2,2017-01-03,1043.839966,1.289189,0.032404
3,2017-01-04,1154.729980,4.224804,0.211802
4,2017-01-05,1013.380005,2.948940,0.115502
...,...,...,...,...
2867,2024-11-07,75904.858462,2.137805,0.935908
2868,2024-11-08,76545.476643,1.705348,0.748932
2869,2024-11-09,76778.867752,1.238379,0.510054
2870,2024-11-10,80474.185602,2.392486,0.350798


# Regressionn

In [191]:
def add_crisis_dummies(merge_result, periods=None):

    # Định nghĩa khoảng thời gian và COVID

    start_covid = pd.to_datetime('2020-01-05')
    end_covid = pd.to_datetime('2023-05-07')

    for asset_name, df in merge_result[periods].items():
        # Kiểm tra nếu cột 'Date' tồn tại
        if 'Date' not in df.columns:
            print(f"Dữ liệu của {asset_name} không có cột 'Date'. Bỏ qua.")
            continue

        # Đảm bảo cột 'Date' là kiểu datetime
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df.dropna(subset=['Date'], inplace=True)

        # Tạo biến giả  COVID nếu dữ liệu đủ khoảng thời gian
        
        df['COVID'] = df['Date'].between(start_covid, end_covid).astype(int) if df['Date'].min() <= start_covid and df['Date'].max() >= end_covid else None

    
    return merge_result



In [192]:
regression_data=add_crisis_dummies(merge_result,'Full sample')

In [193]:
regression_crisis_data=add_crisis_dummies(merger_result_crisis,'COVID-19')

In [194]:

import statsmodels.api as sm
from arch import arch_model


def create_dummy_variables_for_extreme_downturns(returns, quantiles=[0.01, 0.025, 0.05],):

    dummies = {}
    for q in quantiles:
        quantile_value = returns.quantile(q)  # Tính phân vị
        # Tạo biến giả: 1 nếu lợi nhuận nhỏ hơn phân vị, 0 nếu ngược lại
        dummies[f'D(r_stock,{int(q*100)})'] = (returns < quantile_value).astype(int)
    
    return pd.DataFrame(dummies)

def run_gjr_garch_with_dummies(data, asset_name):
    """
    Chạy mô hình GJR-GARCH (1,1) cho từng tài sản với biến giả cho các cuộc khủng hoảng.
    """
    required_columns = ['Log Return', 'SP500_Log_Return', 'COVID']
    if not all(col in data.columns for col in required_columns):
        print(f"Dữ liệu của {asset_name} không đủ các cột cần thiết. Bỏ qua.")
        return None, None

    # Chuyển đổi và loại bỏ NaN
    data[required_columns] = data[required_columns].apply(pd.to_numeric, errors='coerce')
    data.dropna(subset=required_columns, inplace=True)

    # Tính toán các biến giả cho các phân vị 1%, 2.5%, và 5%
    dummy_vars = create_dummy_variables_for_extreme_downturns(data['Log Return'])

    # Xác định biến phụ thuộc (y) và biến độc lập (X) bao gồm các biến giả
    y = data['Log Return']
    X = pd.concat([data[['SP500_Log_Return', 'COVID']], dummy_vars], axis=1)
    X = sm.add_constant(X)

    try:
        # Chạy mô hình hồi quy OLS để lấy phần dư
        ols_model = sm.OLS(y, X).fit()
        residuals = ols_model.resid

        # Tái chuẩn hóa nếu cần thiết
        if residuals.abs().max() < 1e-3:
            residuals *= 100

        # Chạy mô hình GJR-GARCH (1,1)
        gjr_garch_model = arch_model(residuals, vol='Garch', p=1, o=1, q=1, dist='t', rescale=False)
        gjr_garch_fit = gjr_garch_model.fit(update_freq=5,disp='off')

        return ols_model, gjr_garch_fit
    except Exception as e:
        print(f"Lỗi khi chạy mô hình cho {asset_name}: {e}")
        return None, None

# Lưu trữ kết quả
regression_garch_fullsample = []
for asset_name, df in regression_data['Full sample'].items():
    if asset_name != "sp500":
        ols_model, garch_model = run_gjr_garch_with_dummies(df, asset_name)
        if ols_model and garch_model:
            coefficients = ols_model.params
            p_values = ols_model.pvalues
            
            # Hàm tạo ký hiệu dựa trên mức ý nghĩa
            def significance_stars(p_value):
                if p_value < 0.01:
                    return "*"  # Ý nghĩa thống kê ở mức 1%
                elif p_value < 0.05:
                    return "**"   # Ý nghĩa thống kê ở mức 5%
                elif p_value < 0.10:
                    return "***"    # Ý nghĩa thống kê ở mức 10%
                else:
                    return ""     # Không có ý nghĩa thống kê
            
            regression_garch_fullsample.append({
                'Variable': asset_name,
                'Hedge (c0)': f"{coefficients['SP500_Log_Return']:.4f}{significance_stars(p_values['SP500_Log_Return'])}",
                'COVID-19 dummy (c1)': f"{coefficients['COVID']:.4f}{significance_stars(p_values['COVID'])}"
            })

# Lưu trữ kết quả cho giai đoạn khủng hoảng
regressions_garch_crisis = []
for asset_name, df in regression_crisis_data['COVID-19'].items():
    if asset_name != "sp500":
        ols_model, garch_model = run_gjr_garch_with_dummies(df, asset_name)
        if ols_model and garch_model:
            coefficients = ols_model.params
            p_values = ols_model.pvalues
            
            # Tính toán phân vị từ volatility của mô hình GARCH
            volatility = garch_model.conditional_volatility
            c1_quantile = np.percentile(volatility, 1)
            c2_quantile = np.percentile(volatility, 2.5)
            c3_quantile = np.percentile(volatility, 5)
            
            regressions_garch_crisis.append({
                'Variable': asset_name,
                'Hedge (c0)': f"{coefficients['SP500_Log_Return']:.4f}{significance_stars(p_values['SP500_Log_Return'])}",
                '1% quantile (c1)': f"{c1_quantile:.4f}{significance_stars(c1_quantile)}",
                '2.5% quantile (c2)': f"{c2_quantile:.4f}{significance_stars(c2_quantile)}",
                '5% quantile (c3)': f"{c3_quantile:.4f}{significance_stars(c3_quantile)}"
            })



In [195]:
# Chuyển kết quả thành DataFrame và hiển thị
fullsample_df_garch = pd.DataFrame(regression_garch_fullsample)

In [196]:
fullsample_df_garch

,Variable,Hedge (c0),COVID-19 dummy (c1)
0,gold,0.0358*,0.0194*
1,silver,0.2030*,0.0707*
2,dollar,-0.0864*,0.0088*
3,imus,0.0348*,0.0172*
4,wti,0.3454*,0.1652*
5,gsci,0.0348*,0.0172*
6,franc,0.0366*,0.0172*
7,bitcoin,0.5082*,-0.0208
8,ethereum,0.8352*,0.1115**


In [197]:
# Chuyển kết quả thành DataFrame và hiển thị
crisis_df_garch = pd.DataFrame(regressions_garch_crisis)

In [198]:
crisis_df_garch

,Variable,Hedge (c0),1% quantile (c1),2.5% quantile (c2),5% quantile (c3)
0,gold,0.0765*,0.0938***,0.0940***,0.0943***
1,silver,0.2782*,0.1900,0.1904,0.1912
2,dollar,-0.1312*,0.0381**,0.0385**,0.0390**
3,imus,0.0694*,0.0490**,0.0491**,0.0493**
4,wti,0.3774*,0.2298,0.2328,0.2380
5,gsci,0.0694*,0.0490**,0.0491**,0.0493**
6,franc,0.0657*,0.0463**,0.0463**,0.0466**
7,bitcoin,0.8911*,0.3518,0.3532,0.3559
8,ethereum,1.1875*,0.4639,0.4646,0.4664
